# Introduction

Imitation learning is a type of machine learning where an agent learns to imitate the actions of an expert. In order to train a robust agent, we need a large amount of data. However, manual data collection through human demonstrations is time-consuming and expensive.

Isaac Lab Mimic is a feature included in Isaac Lab that allows users to generate new demonstrations by synthesizing trajectories using a small number of human demonstrations. This blueprint will show how to use Isaac Lab Mimic to generate new motion trajectories for a Franka robotic arm and then visually augment using NVIDIA Cosmos to create datasets for imitation learning. The workflow is broken down into two main steps:

1. Generate new demonstrations by synthesizing trajectories using a small number of human demonstrations with Isaac Lab Mimic.
2. Apply diverse visual transformations using NVIDIA Cosmos to the new demonstrations to create a large and diverse dataset.

This notebook will guide you through each step of the workflow.

**NOTE: This notebook must be run on the same machine as the Isaac Sim simulator and a display must be connected to the machine.**


# Understanding the Blueprint

## Motion Trajectory Synthesis
Isaac Lab Mimic is a feature set bundled with Isaac Lab (an open source robotic learning framework designed to help train robot policies). The core idea of Mimic is to allow users to synthetically generate a large number of new robot motion trajectories using only a handful of human demonstrations, thus greatly reducing the amount of time and effort required to collect a dataset for imitation learning. 

Human datasets are annotated with subtask information, which Isaac Lab Mimic uses to construct trajectories for new scene configurations by spatially transforming the original demonstrations.

## Visual Augmentation
Once generated, the new motion trajectories can be visually augmented using NVIDIA Cosmos to create a diverse dataset that is suitable for training an imitation learning policy. 

By using multi-staged data generation scheme, we can automatically create a robust dataset for training complex imitation learning policies without the need for large amounts of manual human data, greatly increasing the amount of data available for training and lowering the amount of time required to collect a dataset.


# Generate a New Motion Trajectory


## Setup Initial Configuration for Isaac Lab

This cell sets up the basic configuration for data generation:

1. **How to Modify**:
   - Adjust `num_envs` based on your GPU capability
   - Set `generation_num_trials` to how many successful trials to execute. Note that some trials may be unsuccessful and so the total number of trials performed may be larger.

2. **Tips**:
   - Start with 1 trial for testing, increase for training. Increasing trails will increase the time it takes to generate the dataset.

In [1]:
from notebook_widgets import create_num_trials_input

num_envs = 1
num_trials = create_num_trials_input()

HTML(value='<h3>Set Number of Trials</h3>')

HTML(value='<p>How many demonstrations to generate</p>')

BoundedIntText(value=1, description='Number of Trials:', layout=Layout(width='300px'), min=1, style=Descriptio…

## Spin up the Simulation

Run this cell to start the simulation environment. This sets up the necessary components for data generation.

**NOTE**: When the simulation is running, a **"Isaac Sim" is not responding."** pop up may appear. This is expected. Please click the **Wait** option and wait while the process completes.

In [2]:
# ============================================================
# [선택] 이 셀은 Interactive Parameter Updates를 사용할 때만 실행
# Data Generation만 필요하면 이 셀을 건너뛰세요 (다음 셀로 이동)
# ============================================================
SKIP_SIMULATION = True  # False로 변경하면 Simulation 실행

if SKIP_SIMULATION:
    print("=" * 60)
    print("⚠️  Simulation 셀 스킵됨 (SKIP_SIMULATION = True)")
    print("=" * 60)
    print("💡 Data Generation 셀로 이동하세요")
    print("   (Interactive Parameter Updates도 스킵)")
    print("")
    print("📌 Interactive Parameters를 사용하려면:")
    print("   1. 위에서 SKIP_SIMULATION = False 로 변경")
    print("   2. Kernel -> Restart 후 다시 실행")
else:
    #-------------------------------------------------------------------------------------------------------------------------------------------------
    # (260130) Added by yjchoi

    # [Blackwell GPU 호환성 패치 - 최종의 최종]
    import sys
    import os
    import glob

    # 1. [경로 선점] 우리가 설치한 'pip_overrides'를 최우선 순위로 등록
    POD = "/workspace/isaaclab/pip_overrides"
    if POD not in sys.path:
        sys.path.insert(0, POD)

    # 2. [강력 차단] Isaac Sim이 몰래 심어놓은 구버전(pip_prebundle) 경로 제거
    sys.path = [p for p in sys.path if "pip_prebundle" not in p]

    # 3. [환경 변수 주입] 라이브러리 경로 강제 설정
    os.environ["PYTHONPATH"] = POD + ":" + os.environ.get("PYTHONPATH", "")
    libs = glob.glob(f"{POD}/nvidia/*/lib")
    libs.append(f"{POD}/torch/lib")
    os.environ["LD_LIBRARY_PATH"] = ":".join(libs) + ":" + os.environ.get("LD_LIBRARY_PATH", "")

    print("🚀 라이브러리 경로 설정 완료. 중요 모듈 선점 로딩 시작...")

    # 4. [모듈 박제] AppLauncher가 실행되기 전에 올바른 버전들을 미리 로드합니다.
    import torch
    import torchvision  # <--- 이 친구가 다시 들어갔습니다! (필수)
    import numpy as np
    import scipy

    # 5. [검증] 눈으로 확인
    print(f"✅ Loaded Torch Ver: {torch.__version__}")
    print(f"✅ Loaded Vision Ver: {torchvision.__version__}")
    print(f"✅ Loaded Numpy Ver: {np.__version__}")  # 1.26.4 여야 함
    print(f"✅ Loaded Scipy Ver: {scipy.__version__}") # 1.12.0 여야 함

    # Vision이 우리 폴더에서 로드되었는지 확인
    if "pip_overrides" in torchvision.__file__:
        print("🎉 성공! 올바른 Torchvision이 로드되었습니다.")
    else:
        print(f"🔥 경고! Torchvision이 엉뚱한 곳에서 로드됨: {torchvision.__file__}")
    #-------------------------------------------------------------------------------------------------------------------------------------------------

    import os
    import nest_asyncio
    nest_asyncio.apply()

    from argparse import ArgumentParser, Namespace
    from isaaclab.app import AppLauncher

    parser = ArgumentParser()
    AppLauncher.add_app_launcher_args(parser)
    args_cli = parser.parse_args([])
    args_cli.enable_cameras = True
    args_cli.kit_args = "--enable omni.videoencoding"

    #-------------------------------------------------------------------------------------------------------------------------------------------------
    # (260202) Added by yjchoi

    # [수정 전]
    # args_cli.headless = False 

    # [수정 후: 화면 없이 계산만 하겠다]
    args_cli.headless = True
    #-------------------------------------------------------------------------------------------------------------------------------------------------

    config = {
        "task": "Isaac-Stack-Cube-Franka-IK-Rel-Blueprint-Mimic-v0",  
        "num_envs": num_envs,                                       
        "generation_num_trials": num_trials.value,                         
        "input_file": "datasets/annotated_dataset.hdf5",     
        "output_file": "datasets/generated_dataset.hdf5", 
        "pause_subtask": False,
        "enable": "omni.kit.renderer.capture",
    }

    # Update the default configuration
    args_dict = vars(args_cli)
    args_dict.update(config)
    args_cli = Namespace(**args_dict)

    # Now launch the simulator with the final configuration
    app_launcher = AppLauncher(args_cli)
    simulation_app = app_launcher.app

    import asyncio
    import gymnasium as gym
    import numpy as np
    import random
    import torch

    import isaaclab_mimic.envs  # noqa: F401
    from isaaclab_mimic.datagen.generation import env_loop, setup_env_config, setup_async_generation
    from isaaclab_mimic.datagen.utils import get_env_name_from_dataset, setup_output_paths, interactive_update_randomizable_params, reset_env
    from isaaclab.managers import ObservationTermCfg as ObsTerm
    from notebook_utils import ISAACLAB_OUTPUT_DIR

    import isaaclab_tasks  # noqa: F401
    num_envs = args_cli.num_envs

    # Setup output paths and get env name
    output_dir, output_file_name = setup_output_paths(args_cli.output_file)
    env_name = args_cli.task or get_env_name_from_dataset(args_cli.input_file)

    # Configure environment
    env_cfg, success_term = setup_env_config(
        env_name=env_name,
        output_dir=output_dir,
        output_file_name=output_file_name,
        num_envs=num_envs,
        device=args_cli.device,
        generation_num_trials=args_cli.generation_num_trials,
    )
    # Set observation output directory
    for obs in vars(env_cfg.observations.rgb_camera).values():
        if not isinstance(obs, ObsTerm):
            continue
        obs.params["image_path"] = os.path.join(ISAACLAB_OUTPUT_DIR, obs.params["image_path"])
    env_cfg.observations


    # create environment
    env = gym.make(env_name, cfg=env_cfg).unwrapped

    # set seed for generation
    random.seed(env.cfg.datagen_config.seed)
    np.random.seed(env.cfg.datagen_config.seed)
    torch.manual_seed(env.cfg.datagen_config.seed)

    # reset before starting
    reset_env(env, 100)

⚠️  Simulation 셀 스킵됨 (SKIP_SIMULATION = True)
💡 Data Generation 셀로 이동하세요
   (Interactive Parameter Updates도 스킵)

📌 Interactive Parameters를 사용하려면:
   1. 위에서 SKIP_SIMULATION = False 로 변경
   2. Kernel -> Restart 후 다시 실행


## Interactive Parameter Updates

To get diversity in the generated motion trajectories, the scene configuration is randomized for each trial. This section provides interactive sliders and controls to adjust various environment parameters in real-time:

1. **What You'll See**:
   - Sliders for numerical values
   - Range inputs for min/max settings
   - Current value displays
   - Parameter names and allowed ranges

2. **How to Use**:
   - Move the sliders to adjust values
   - Watch the environment update in real-time

3. **Available Parameters**:
   - **Franka Joint State Randomization**:
     - **mean (0.0 - 0.5)**: Controls the average joint angle offset (in radians)
     - **std (0.0 - 0.1)**: Controls the spread of randomization around the mean

   - **Cube Position Randomization**:
     - **pose_range.x (0.3 - 0.9)**: Controls cube placement along the x-axis (in meters)
     - **pose_range.y (-0.3 - 0.3)**: Controls cube placement along the y-axis (in meters)
     - **min_separation (0.0 - 0.5)**: Minimum allowed distance between cubes (in meters)
     
     **Note:** If the system cannot place cubes with the specified minimum separation after several attempts (due to space constraints), it will accept the last generated positions even if they don't meet the separation requirement. This prevents the system from getting stuck in an impossible configuration.


4. **Tips**:
   - Start with small adjustments to understand their effects

Note: These adjustments will affect how new demonstrations are generated, so take time to experiment with different settings to achieve desired behavior.

In [3]:
# ============================================================
# [선택] 이 셀은 Interactive Parameter Updates를 사용할 때만 실행
# Data Generation만 필요하면 이 셀을 건너뛰세요 (다음 셀로 이동)
# ============================================================
SKIP_SIMULATION = True  # False로 변경하면 Simulation 실행

if not SKIP_SIMULATION:
    randomizable_params = {
        "randomize_franka_joint_state": {
            "mean": (0.0, 0.5, 0.01),
            "std": (0.0, 0.1, 0.01),
        },
        "randomize_cube_positions": {
            "pose_range": {
                    "x": (0.3, 0.9, 0.01),
                    "y": (-0.3, 0.3, 0.01),
                },
            "min_separation": (0.0, 0.5, 0.01),
        }
    }
    
    for i in range(len(env.unwrapped.event_manager._mode_term_cfgs["reset"])):
        event_term = env.unwrapped.event_manager._mode_term_cfgs["reset"][i]
        name = env.unwrapped.event_manager.active_terms["reset"][i]
        display(f"Updating parameters for event: {event_term.func.__name__}")
        interactive_update_randomizable_params(event_term, name, randomizable_params[name], env=env)


## Data Generation

Run this cell to start generating demonstrations using the parameters you've configured. The process will:
- Generate the specified number of demonstrations
- Save successful demonstrations to your output file
- Show progress as demonstrations are generated

In [4]:
# ============================================================
# v17: Standalone 전용 Data Generation (3가지 개선)
# - NUM_TRIALS 환경 변수 명령어에 직접 전달
# - pip_overrides 디렉토리에 패키지 설치
# ============================================================
import os
import sys

print("=" * 60)
print("[v17] Data Generation - Standalone 전용")
print("=" * 60)

# 환경 변수 값 준비
#trials = num_trials.value
trials = 3

print(f"[설정] 목표 생성 횟수: {trials}")
print("-" * 60)

# 필수 패키지를 pip_overrides 디렉토리에 설치
print("[1/2] 필수 패키지 설치 중 (pip_overrides)...")
sys.stdout.flush()
install_code = os.system(
    "/isaac-sim/kit/python/bin/python3 -m pip install "
    "--target=/workspace/isaaclab/pip_overrides "
    "nest_asyncio toml -q"
)
if install_code != 0:
    print(f"[WARNING] 패키지 설치 실패 (코드: {install_code}), 계속 진행...")

print("[2/2] standalone 스크립트 실행 중...")
print("(출력이 여기에 표시됩니다)")
print("-" * 60)
sys.stdout.flush()

# 환경 변수를 명령어에 직접 전달
return_code = os.system(
    f"cd /workspace/isaaclab && NUM_TRIALS={trials} "
    f"./_isaac_sim/python.sh -u generate_data_standalone.py"
)

if return_code != 0:
    print(f"\n[ERROR] 종료 코드: {return_code}")
else:
    print("\n[SUCCESS] 데이터 생성 완료!")

[v17] Data Generation - Standalone 전용
[설정] 목표 생성 횟수: 3
------------------------------------------------------------
[1/2] 필수 패키지 설치 중 (pip_overrides)...
[2/2] standalone 스크립트 실행 중...
(출력이 여기에 표시됩니다)
------------------------------------------------------------



[notice] A new release of pip is available: 24.0+nv1 -> 26.0
[notice] To update, run: /isaac-sim/kit/python/bin/python3 -m pip install --upgrade pip


[Standalone] Data Generation 시작 (v5 - nest_asyncio)
PyTorch: 2.11.0.dev20260201+cu128
Torchvision: 0.25.0.dev20260201+cu128
[WARN][AppLauncher]: There are no arguments attached to the ArgumentParser object. If you have your own arguments, please load your own arguments before calling the `AppLauncher.add_app_launcher_args` method. This allows the method to check the validity of the arguments and perform checks for argument names.
[설정] 목표 생성 횟수: 3
[1/5] AppLauncher 시작 중...
[INFO][AppLauncher]: Loading experience file: /workspace/isaaclab/apps/isaaclab.python.headless.rendering.kit
[Warning] [simulation_app.simulation_app] fast shutdown not supported with jupyter notebooks
Loading user config located at: '/isaac-sim/kit/data/Kit/Isaac-Sim/4.5/user.config.json'
[Info] [carb] Logging to file: /isaac-sim/kit/logs/Kit/Isaac-Sim/4.5/kit_20260204_160851.log
2026-02-04 07:08:51 s] [Warning] [omni.kit.app.plugin] No crash reporter present, dumps uploading isn't available.
2026-02-04 07:08:51 s] 

Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified



|---------------------------------------------------------------------------------------------|
| Driver Version: 580.126.09    | Graphics API: Vulkan
|=============================================================================================|
| GPU | Name                             | Active | LDA | GPU Memory | Vendor-ID | LUID       |
|     |                                  |        |     |            | Device-ID | UUID       |
|     |                                  |        |     |            | Bus-ID    |            |
|---------------------------------------------------------------------------------------------|
| 0   | NVIDIA RTX PRO 6000 Blackwell .. | Yes: 0 |     | 97887   MB | 10de      | 0          |
|     |                                  |        |     |            | 2bb4      | c0943997.. |
|     |                                  |        |     |            | 1         |            |
|===============================================================================

Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified


2026-02-04 07:08:52 [1,147ms] [Warning] [carb.windowing-glfw.plugin] GLFW initialization failed.
2026-02-04 07:08:52 [1,147ms] [Warning] [carb] Failed to startup plugin carb.windowing-glfw.plugin (interfaces: [carb::windowing::IGLContext v1.0],[carb::windowing::IWindowing v1.5]) (impl: carb.windowing-glfw.plugin)
2026-02-04 07:08:52 [1,158ms] [Warning] [carb.windowing-glfw.plugin] GLFW initialization failed.
2026-02-04 07:08:52 [1,158ms] [Warning] [carb] Failed to startup plugin carb.windowing-glfw.plugin (interfaces: [carb::windowing::IGLContext v1.0],[carb::windowing::IWindowing v1.5]) (impl: carb.windowing-glfw.plugin)
2026-02-04 07:08:52 [1,163ms] [Warning] [carb.windowing-glfw.plugin] GLFW initialization failed.
2026-02-04 07:08:52 [1,163ms] [Warning] [carb] Failed to startup plugin carb.windowing-glfw.plugin (interfaces: [carb::windowing::IGLContext v1.0],[carb::windowing::IWindowing v1.5]) (impl: carb.windowing-glfw.plugin)
2026-02-04 07:08:52 [1,168ms] [Warning] [carb.windowing

# Cosmos

Now that a new motion trajectory has been generated, we will apply visual transformations to the data using Cosmos to create a realistic demo that is suitable for training an imitation learning policy.

## Video Preprocessing
In this first step, we will process the generated motion trajectory into a video that can be used as an input for the Cosmos model.
The normals of the scene are used to apply shading to the semantic segmentation which produces an input that works very well with the Cosmos model.

In [5]:
from notebook_widgets import create_camera_input
from notebook_utils import ISAACLAB_OUTPUT_DIR

VIDEO_LENGTH = 120   # Suggested length is between 120 and 200
camera_selection = create_camera_input(ISAACLAB_OUTPUT_DIR)

HTML(value='<h3>1. Select Camera</h3>')

Dropdown(description='Source Camera:', layout=Layout(width='auto'), options=('table_cam', 'table_high_cam'), s…

In [6]:
# ============================================================
# v20: Video Encoding - Standalone 방식
# - Isaac Sim 환경에서 GPU 가속 비디오 인코딩 실행
# - os.system()으로 encode_video_standalone.py 호출
# ============================================================
import os
import sys
import json
from IPython.display import Video
from notebook_utils import VIDEO_OUTPUT_DIR

print("=" * 60)
print("[v20] Video Encoding - Standalone 방식")
print("=" * 60)

# 비디오 출력 폴더 생성
os.makedirs(VIDEO_OUTPUT_DIR, exist_ok=True)

# 비디오 설정
video_config = {
    "camera": camera_selection.value,
    "video_length": VIDEO_LENGTH
}

print(f"[설정] 카메라: {video_config['camera']}")
print(f"[설정] 비디오 길이: {video_config['video_length']} 프레임")
print("-" * 60)
print("standalone 스크립트 실행 중...")
print("(출력이 여기에 표시됩니다)")
print("-" * 60)
sys.stdout.flush()

# standalone 스크립트 실행
return_code = os.system(
    f"cd /workspace/isaaclab && "
    f"VIDEO_CONFIG='{json.dumps(video_config)}' "
    f"./_isaac_sim/python.sh -u encode_video_standalone.py"
)

if return_code != 0:
    print(f"\n[ERROR] 종료 코드: {return_code}")
else:
    print("\n[SUCCESS] 비디오 인코딩 완료!")

    # 생성된 비디오 표시
    camera = video_config['camera']
    if os.path.isdir(VIDEO_OUTPUT_DIR):
        for f in sorted(os.listdir(VIDEO_OUTPUT_DIR)):
            if f.startswith(camera) and f.endswith('.mp4'):
                video_path = os.path.join(VIDEO_OUTPUT_DIR, f)
                print(f"\n{video_path}")
                display(Video(video_path, width=800))

[v20] Video Encoding - Standalone 방식
[설정] 카메라: table_cam
[설정] 비디오 길이: 120 프레임
------------------------------------------------------------
standalone 스크립트 실행 중...
(출력이 여기에 표시됩니다)
------------------------------------------------------------
[Standalone] Video Encoding 시작
PyTorch: 2.11.0.dev20260201+cu128
[1/4] AppLauncher 시작 중...
[WARN][AppLauncher]: There are no arguments attached to the ArgumentParser object. If you have your own arguments, please load your own arguments before calling the `AppLauncher.add_app_launcher_args` method. This allows the method to check the validity of the arguments and perform checks for argument names.
[INFO][AppLauncher]: Loading experience file: /workspace/isaaclab/apps/isaaclab.python.headless.kit
[Warning] [simulation_app.simulation_app] fast shutdown not supported with jupyter notebooks
Loading user config located at: '/isaac-sim/kit/data/Kit/Isaac-Sim/4.5/user.config.json'
[Info] [carb] Logging to file: /isaac-sim/kit/logs/Kit/Isaac-Sim/4.5/kit_2026

Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified


2026-02-04 07:17:15 s] [Warning] [omni.isaac.dynamic_control] omni.isaac.dynamic_control is deprecated as of Isaac Sim 4.5. No action is needed from end-users.


2026-02-04 07:17:16 [1,219ms] [Error] [omni.ext._impl._internal] Failed to import python module isaaclab_tasks from /workspace/isaaclab/source/isaaclab_tasks. Error: operator torchvision::nms does not exist. Traceback:
Traceback (most recent call last):
  File "/workspace/isaaclab/_isaac_sim/kit/kernel/py/omni/ext/_impl/_internal.py", line 187, in _custom_importer
    return _import_public(ext_module.path, ext_module.name, reload_enabled)
  File "/workspace/isaaclab/_isaac_sim/kit/kernel/py/omni/ext/_impl/_internal.py", line 94, in _import_public
    module = import_module(module_name)
  File "/workspace/isaaclab/_isaac_sim/kit/kernel/py/omni/ext/_impl/custom_importer.py", line 76, in import_module
    return importlib.import_module(name)
  File "/workspace/isaaclab/_isaac_sim/kit/python/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  Fil


|---------------------------------------------------------------------------------------------|
| Driver Version: 580.126.09    | Graphics API: Vulkan
|=============================================================================================|
| GPU | Name                             | Active | LDA | GPU Memory | Vendor-ID | LUID       |
|     |                                  |        |     |            | Device-ID | UUID       |
|     |                                  |        |     |            | Bus-ID    |            |
|---------------------------------------------------------------------------------------------|
| 0   | NVIDIA RTX PRO 6000 Blackwell .. | Yes: 0 |     | 97887   MB | 10de      | 0          |
|     |                                  |        |     |            | 2bb4      | c0943997.. |
|     |                                  |        |     |            | 1         |            |
|===============================================================================


output/videos/table_cam_demo_1.mp4



output/videos/table_cam_demo_2.mp4


## Deploying Cosmos
Deploy Cosmos on your provider of choice, or to your own local resources: [Cosmos Transfer1](https://huggingface.co/nvidia/Cosmos-Transfer1-7B). 
Click on the `Code` link on the Cosmos Transfer page and follow the installation steps outlined in the README. You can find detailed setup instructions in under `examples/inference_multi_control_manual_input.md`.

> ### Adding a Web API to Cosmos Transfer1
> To simplify testing, copy the file `notebook/app.py` into the Cosmos root directory, and run it with `python app.py`. This will expose endpoints which we'll use to communicate between the notebook and the cosmos model. The script exposes endpoints at port `5000` by default.

In [7]:
import ipywidgets as widgets
url_widget = widgets.Text(value="", placeholder="cosmos/url:port", description="Cosmos URL:", style={'description_width': 'initial'}, layout={"width": "1000px"})
display(url_widget)


Text(value='', description='Cosmos URL:', layout=Layout(width='1000px'), placeholder='cosmos/url:port', style=…

### Using the Cosmos Model

The Cosmos model has several available parameters which alter the output in various ways:
- `prompt`: Text prompt for the video generation.
- `seed`: Seed for the random number generator. `int [0 - 2147483648]`
- `control_weight`: Controls how strongly the control input should affect the output. The stronger the effect, the more adherance to the control input, but the less the model generation freedom. `float [0 - 1.0]`
- `sigma_max`: A float value representing the maximum sigma. Lower values result in less change from the original input while a larger values allows for more change but may diverge more from the input scene. `float [0 - 80.0]`

In [8]:
from notebook_widgets import create_variable_dropdowns, create_cosmos_params
from notebook_utils import VIDEO_OUTPUT_DIR, COSMOS_OUTPUT_DIR

prompt_manager = create_variable_dropdowns("stacking_prompt.toml")
cosmos_params = create_cosmos_params(VIDEO_OUTPUT_DIR)  # v19: 비디오 폴더에서 선택

HTML(value='<h3>Prompt Generator</h3>')

HTML(value='<h4>Prompt</h4>')

HTML(value='The scene depicts a robotic arm performing a precise block-stacking operation with glass\ncubes in…

HTML(value='<h3>Cosmos Parameters</h3>')

Dropdown(description='Input Video:', layout=Layout(width='500px'), options=('table_cam_demo_0.mp4', 'table_cam…

## Generate with Cosmos
---
> **NOTE:** Generation generally takes around 5 to 10 minutes on a single H100 GPU depending on the video length.

---

> **Tips:**
> - To increase prompt adherence, try increasing the `Sigma Max` value
> - To reduce divergence from the input scene, try increasing the `Control Weight` and/or increasing `Canny Strength`

In [9]:
import os
from cosmos_request import process_video
from notebook_utils import VIDEO_OUTPUT_DIR, COSMOS_OUTPUT_DIR
from notebook_widgets import create_download_link
from IPython.display import Video, clear_output

# Cosmos 출력 폴더 생성
os.makedirs(COSMOS_OUTPUT_DIR, exist_ok=True)

params = {k: w.value for k, w in cosmos_params.items()}
video_filepath = os.path.join(VIDEO_OUTPUT_DIR, params.pop("input_video"))  # v19: 비디오 폴더에서 읽기
output_path = os.path.join(COSMOS_OUTPUT_DIR, f"cosmos_{params['seed']}.mp4")
params["prompt"] = prompt_manager.prompt

if not url_widget.value:
    raise ValueError("Enter URL to proceed.")

response = process_video(
    url=url_widget.value,
    video_path=video_filepath,
    output_path=output_path,
    **params,
)
if response is None:
    display("An error occurred processing the request")
elif response.status_code == 200:
    clear_output(wait=True)
    display(Video(output_path))
    display(create_download_link(output_path, link_text=f"Download Video: {output_path}"))

ValueError: Enter URL to proceed.